In [1]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier

from csv import writer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import tree
import matplotlib.pyplot as plt
import time

# Mapping Function (Encoding of attributes)

In [2]:
#grp_gap ==> maximum number of allowed attribute values under each group. 
#It is initialized as 20. However, we can provide any value here

# grp is a dict of attribute groups used during mapping/encoding

#map_type == 1 >>> normal encoding
#map_type == 2 >>> encoding based on atrribute group



def GetAttributeMapping(data,grp=None,grp_gap=20,map_type=1):
    mapping = {}
    mapping['NotA'] = -1
    mapping[0] = 0
    mapping['YES'] = 1
    mapping['NO'] = 0
    if(map_type==1):
        for col in data.columns[:4]:
            #print(col,it_full[col].unique())
            col_un = data[col].unique()
            cnt = 1
            for val in col_un:
                if(val != 'NotA'):
                    mapping[val] = cnt
                    cnt = cnt + 1
        return mapping
    elif(map_type==2):
        for col in data.columns[1:3]:
            #print(col,it_full[col].unique())
            col_un = data[col].unique()
            cnt = 1
            for val in col_un:
                if(val != 'NotA'):
                    mapping[val] = cnt
                    cnt = cnt + 1
        
        for g in grp:
            grp_num = 1
            for member in grp[g]:
                mem_num = 1
                for val in member:
                    mapping[val] = grp_num*grp_gap+mem_num
                    mem_num = mem_num + 1
                grp_num = grp_num + 1
        return mapping
        
    

# Create groups of attributes

In [3]:
designation_grp = [['CEO'],
                   ['CTO'],
                   ['FINANCE MANAGER'],
                   ['HR MANAGER'],
                   ['DESIGNER','PROGRAMMER','SDE','TESTER'],
                   ['PROJECT_MANAGER','SYSTEM_ARCHITECT'],
                   ['PROJECT_LEADER','PRINCIPAL'],
                   ['IT_MANAGER','SECURTY_ENGINEER'],
                   ['NETWORK_ENGINEER'],
                   ['DATABASE_ENGINEER']
                  ]

resource_grp = [['EMP_DETAIL'],
                ['CLIENT_DETAIL'],
                ['SALARY_DETAIL','PF_DETAIL'],
                ['PROJECT_DETAIL','PROJECT_PLAN'],
                ['NETWORK_SETUP'],
                ['DATABASE'],
                ['PROJECT_COST','ALLOCATED_FUND','FINANCE_REPORT','TAX_DETAIL'],
                ['SERVER','STORAGE','GPU']
               ]

attr_grp={}
attr_grp['DESIGNATION'] = designation_grp
attr_grp['Resource'] = resource_grp


# Find Relation between common subject and object attributes 

In [4]:
def same_conditions(col1,col2):
    if ((col1 == -1) or (col2 == -1)):
        return 2
    elif (col1==col2):
        return 1
    else:
        return 0
def chk_nota(col):
    if(col==-1):
        return 1
    else:
        return 0

# Prepare the data for Training and Testing based on relation

In [5]:
def GetPreparedData(train_data,test_data,prep_type=4):
    data = pd.concat([train_data,test_data],axis=0)
    #print(data.columns)
    #exit(0)
    if(prep_type==1):#Naive
        map_type = 1
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        print(mapping)
        data_encoded = data.replace(mapping)
    elif(prep_type==2):#Columns for same attribute values in subject and object
        map_type = 1
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        data_encoded = data.replace(mapping)
        data_encoded['sameProj'] = data_encoded.apply(lambda x: same_conditions(x['Project_name'], x['Project_Name']), axis=1)
        data_encoded['sameDep'] = data_encoded.apply(lambda x: same_conditions(x['Department'], x['Department.1']), axis=1)
        data_encoded = data_encoded.drop('Department',axis=1)
        data_encoded = data_encoded.drop('Department.1',axis=1)
        data_encoded = data_encoded.drop('Project_name',axis=1)
        data_encoded = data_encoded.drop('Project_Name',axis=1)
    elif(prep_type==3):#Grouping of attributes
        map_type = 2
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        data_encoded = data.replace(mapping)
    elif(prep_type==4):#Grouping of attributes + Columns for same attribute values in subject and object
        map_type = 2
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        data_encoded = data.replace(mapping)
        data_encoded['sameProj'] = data_encoded.apply(lambda x: same_conditions(x['Project_name'], x['Project_Name']), axis=1)
        data_encoded['sameDep'] = data_encoded.apply(lambda x: same_conditions(x['Department'], x['Department.1']), axis=1)
        data_encoded = data_encoded.drop('Department',axis=1)
        data_encoded = data_encoded.drop('Department.1',axis=1)
        data_encoded = data_encoded.drop('Project_name',axis=1)
        data_encoded = data_encoded.drop('Project_Name',axis=1)
    elif(prep_type==5):#Naive+NACol
        map_type = 1
        mapping = GetAttributeMapping(data,grp=attr_grp,map_type=map_type)
        data_encoded = data.replace(mapping)
        data_encoded['Proj_NA'] = data_encoded.apply(lambda x: chk_nota(x['Project_name']), axis=1)
        
        
    
    #X = data_encoded.iloc[:,:-1]
    #y = data_encoded.iloc[:,-1]
    
    X = data_encoded.loc[:, data_encoded.columns != 'Access']
    y = data_encoded.loc[:, data_encoded.columns == 'Access']
    
    X_train = X.head(train_data.shape[0])
    X_test = X.tail(test_data.shape[0])
    y_train = y.head(train_data.shape[0])
    y_test = y.tail(test_data.shape[0])
    #print(X)
    return X_train, X_test, y_train, y_test
        

In [6]:
def score(Ytest, pred, label, clfname, fname,):
    acc = accuracy_score(Ytest, pred)
    pre = precision_score(Ytest, pred)
    rec = recall_score(Ytest, pred)
    f1 = f1_score(Ytest, pred)
    print(f"{label}:\naccuracy: {acc:.2f}\nprecision: {pre:.2f}\nrecall: {rec:.2f}\nfscore: {f1:.2f}\n")
    #scores = [str(clfname),str(label),str(acc),str(pre),str(rec),str(f1)]
    #file_name = './results_new/' + fname + '.csv'
    f = open(fname, "a+")
    #f.write('%s,%s,%0.3f,%f,%f,%f\n' % (clfname,label,acc,pre,rec,f1))
    f.write('%0.3f,%0.3f,%0.3f,%0.3f,' % (acc,pre,rec,f1))
    f.close()
        
    


def record_misclassifications(Xtest, Ytest, pred, fname):
    #fields = ["S-Designation", "S-Department", "S-Degree", "S-Year", "O-Type", "O-Department", "O-Degree", "O-Year", "True_Perm", "Pred_Perm"]
    fields = ['S-Designation', 'S-Project', 'S-Dept', 'O-Res','O-Project', 'O-Dept','true_perm','pred_perm']
    #fields = ["SDesignation", "SDepartment", "SDegree", "SYear", "OType", "ODepartment", "ODegree", "OYear", 'dept', 'deg', 'yr',"True_Perm", "Pred_Perm"]
    misclassified_pts = [(x + [t] + [y]) for (x, t, y) in zip(Xtest, Ytest, pred) if (y != t)]
    with open(f"./results_new/{fname}_misclassications_it_test.csv", 'w') as csvfile:
        csv_writer = writer(csvfile)
        csv_writer.writerow(fields)
        csv_writer.writerows(misclassified_pts)

In [7]:

models_dict = {
    #"SupportVectorMachine": SVC(kernel="linear"),
    #"LogisticRegression": LogisticRegression(solver="saga", n_jobs=-1),
    "ArtificalNeuralNetwork": MLPClassifier(hidden_layer_sizes=30, max_iter=1500),
    "DL": MLPClassifier(hidden_layer_sizes=30, max_iter=1500),
    "DecisionTree": DecisionTreeClassifier(),
    #"ExtraTree": ExtraTreeClassifier(),
    "RandomForest": RandomForestClassifier(n_jobs=-1),
    "ExtraTrees": ExtraTreesClassifier(n_jobs=-1),
    "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="error", n_jobs=-1),
    #"LightGBM": LGBMClassifier(n_estimators=100, n_jobs=-1),
    #"AdaBoost": AdaBoostClassifier(n_estimators=100, learning_rate=1.0),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100, learning_rate=1.0),
}

# Reading Training and Test Data

In [10]:
#fn=['S-Designation', 'O-Res', 'proj','dept']
#cn=['AccessDenied', 'AccessGranted']
file_name = './results_new/' + 'itdataset_overall_result_new1_fclf' + '.csv'
f = open(file_name, "w")
f.write('clf_name,accuracy,precision,recall,f1score,accuracy,precision,recall,f1score,accuracy,precision,recall,f1score,accuracy,precision,recall,f1score,accuracy,precision,recall,f1score,\n')
f.close()

train_file_name = './results_new_train/' + 'itdataset_overall_result_new1_fclf' + '.csv'
tf = open(train_file_name, "w")
tf.write('clf_name,accuracy,precision,recall,f1score,accuracy,precision,recall,f1score,accuracy,precision,recall,f1score,accuracy,precision,recall,f1score,accuracy,precision,recall,f1score,\n')
tf.close()

time_file_name = './results_new/' + 'company_report_time.csv'
ft = open(time_file_name, "w")
ft.write('clf_name,data_preptype,data_preptime,train_time,test_time\n')
ft.close()

#DESIGNATION	Project_name	Department	Resource	Project_Name	Department
fn=['S-Designation', 'S-Project', 'S-Dept', 'O-Res','O-Project', 'O-Dept']
cn=['AccessDenied', 'AccessGranted']

for (name, clf) in models_dict.items():
    tf = open(train_file_name, "a+")
    tf.write('%s,' % (name))
    tf.close()
    f = open(file_name, "a+")
    f.write('%s,' % (name))
    f.close()
    for i in range(1,6):  
        train_data = pd.read_csv('it_train_new1.csv')
        test_data = pd.read_csv('it_test_new1.csv')
        dataprep_start_time = time.time()
        X_train, X_test, y_train, y_test = GetPreparedData(train_data,test_data,prep_type=i)
        
        #print(f"[INFO] Training model: {name}")
        train_start_time = time.time()
        clf.fit(X_train.to_numpy(), y_train.to_numpy())
        train_end_time = time.time()
        #clf.fit(X_it_train_mod_com.to_numpy(), y_it_train_mod.to_numpy())
        #clf.fit(X_test.to_numpy(), Ytest)
        pred = clf.predict(X_test.to_numpy())
        test_end_time = time.time()
        #pred = clf.predict(Xtrain)
        #score(y_train.to_numpy(), clf.predict(X_train.to_numpy()), label="training metrics",clfname=name,fname='itdataset_type4_result')
        #score(y_it_train_mod.to_numpy(), clf.predict(X_it_train_mod.to_numpy()), label="training metrics")
        
        ft = open(time_file_name, "a+")
        ft.write('%s,%d,%f,%f,%f\n' %(name,i,(train_start_time-dataprep_start_time),(train_end_time-train_start_time),(test_end_time-train_end_time)))
        ft.close()
        print('Data_preptime=%f' %(train_start_time-dataprep_start_time))
        print('Training_time=%f' %(train_end_time-train_start_time))
        print('Testing_time=%f' %(test_end_time-train_end_time))
        
        score(y_test.to_numpy(), pred, label=str(i),clfname=name,fname=file_name)
        #score(Ytest, clf.predict(Xtest), label="training metrics")
        pred_train = clf.predict(X_train.to_numpy())
        score(y_train.to_numpy(), pred_train, label=str(i),clfname=name,fname=train_file_name)
        print("-"*60)
        if i==5:
            record_misclassifications(X_test.to_numpy().tolist(), y_test.to_numpy(), pred, fname=name)
        #record_misclassifications(X_it_train_mod_com.to_numpy().tolist(), Y_it_train_mod.to_numpy(), clf.predict(X_it_train_mod_com.to_numpy()), fname=name)
        '''
        if(name=='DecisionTree' and i==1):
            fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (13,13), dpi=600)
            tree.plot_tree(clf,
                       feature_names = fn, 
                       class_names=cn,
                       filled = True);
            figname = name + '_it_train_new1.png'
            fig.savefig(figname)
        '''
    f = open(file_name, "a+")
    f.write('\n')
    f.close()
    
    tf = open(train_file_name, "a+")
    tf.write('\n')
    tf.close()

{'NotA': -1, 0: 0, 'YES': 1, 'NO': 0, 'HR MANAGER': 1, 'FINANCE MANAGER': 2, 'CTO': 3, 'CEO': 4, 'DESIGNER': 5, 'PROGRAMMER': 6, 'SDE': 7, 'PROJECT_MANAGER': 8, 'PROJECT_LEADER': 9, 'NETWORK_ENGINEER': 10, 'DATABASE_ENGINEER': 11, 'IT_MANAGER': 12, 'PRINCIPAL': 13, 'SECURTY_ENGINEER': 14, 'SYSTEM_ARCHITECT': 15, 'TESTER': 16, 'P1': 1, 'P2': 2, 'P4': 3, 'P5': 4, 'P6': 5, 'P7': 6, 'P3': 7, 'HR': 1, 'FINANCE': 2, 'ADMINISTRATION': 3, 'DEVELOPMENT': 4, 'TECHNICAL': 5, 'SALARY_DETAIL': 1, 'CLIENT_DETAIL': 2, 'EMP_DETAIL': 3, 'PROJECT_DETAIL': 4, 'PROJECT_COST': 5, 'SERVER': 6, 'DATABASE': 7, 'NETWORK_SETUP': 8, 'PF_DETAIL': 9, 'FINANCE_REPORT': 10, 'TAX_DETAIL': 11, 'ALLOCATED_FUND': 12, 'STORAGE': 13, 'GPU': 14, 'PROJECT_PLAN': 15}


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Data_preptime=0.025765
Training_time=1.598738
Testing_time=0.000183
1:
accuracy: 0.78
precision: 0.73
recall: 0.56
fscore: 0.63

1:
accuracy: 0.90
precision: 0.84
recall: 0.81
fscore: 0.82

------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Data_preptime=0.124176
Training_time=2.868655
Testing_time=0.000225
2:
accuracy: 0.78
precision: 0.63
recall: 0.79
fscore: 0.70

2:
accuracy: 0.96
precision: 0.93
recall: 0.93
fscore: 0.93

------------------------------------------------------------


...

Data_preptime=0.032441
Training_time=0.114705
Testing_time=0.000687
1:
accuracy: 0.78
precision: 0.68
recall: 0.61
fscore: 0.64

1:
accuracy: 0.94
precision: 0.97
recall: 0.83
fscore: 0.89

------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Data_preptime=0.138101
Training_time=0.080297
Testing_time=0.000624
2:
accuracy: 0.84
precision: 0.72
recall: 0.85
fscore: 0.78

2:
accuracy: 1.00
precision: 1.00
recall: 1.00
fscore: 1.00

------------------------------------------------------------
Data_preptime=0.023796
Training_time=0.093313
Testing_time=0.000578
3:
accuracy: 0.86
precision: 0.91
recall: 0.63
fscore: 0.75

3:
accuracy: 0.94
precision: 0.98
recall: 0.81
fscore: 0.89

------------------------------------------------------------


/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/ensemble/_gb.py:494: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Data_preptime=0.103893
Training_time=0.066806
Testing_time=0.000548
4:
accuracy: 0.99
precision: 1.00
recall: 0.97
fscore: 0.98

4:
accuracy: 1.00
precision: 1.00
recall: 1.00
fscore: 1.00

------------------------------------------------------------
Data_preptime=0.042824
Training_time=0.080187
Testing_time=0.000572
5:
accuracy: 0.78
precision: 0.70
recall: 0.58
fscore: 0.63

5:
accuracy: 0.94
precision: 0.98
recall: 0.81
fscore: 0.89

------------------------------------------------------------
